In [13]:
!pwd

/home/b/b382291/regridder/AQUA


In [31]:
# run mean OPA 
import intake 
import os
# add directory where the utils.py file is located
import xarray as xr 
import importlib
import dask as dk
import numpy as np 

# for levante
os.chdir('/home/b/b382291/git/one_pass/src')
imported_module = importlib.import_module("opa")

os.chdir('/home/b/b382291/git/one_pass/src')
importlib.reload(imported_module)
from opa import *
from opa import opa # from module (file.py) import class name 

os.chdir('/home/b/b382291/regridder/AQUA')
from aqua import Reader
from aqua.reader import catalogue


# initalising the statistic 
stat1 = opa(statistic = "mean", statFreq = "monthly", outputFreq = "weekly", save = "false")

# list of possible statistics: mean, var, std, min, max, threshExceed, hist, percentile



Running the algorithm

In [32]:

for i in range(0, 24*31, 1): 

    # extract moving window 
    ds = tasr.isel(time=slice(i,i+1)) # slice(start, stop, step), extract 'moving window' which is hourly data
    
    dm = stat1.compute(ds)



In [36]:
stat1.timeStamp

Timestamp('2020-01-20 23:00:00')

In [35]:
stat1.timeStamp.day

20

In [10]:
# from dask.distributed import LocalCluster, Client, progress
# processes = False
# nworker = 32
# ncpu = 32
# threads = ncpu // nworker
# print(f"Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}, processes: {processes}")
# client = Client(processes=processes,
#  threads_per_worker=threads,
#  n_workers=nworker,
#  memory_limit='64GB'
#  )

#catalogue();

# REGRIDDER 

os.chdir('/home/b/b382291/regridder/AQUA')

reader = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", regrid="r020")
data = reader.retrieve(fix=False)
tasr = reader.regrid(data["10u"][:,:])

#tasr = reader.regrid(data["2t"][0:16,:])

In [180]:
# checking the output against np.mean 

ds2 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
#npStat = np.mean(ds2, axis = axNum, dtype = np.float64)
#npStat = np.var(ds2, axis = axNum, dtype = np.float64, ddof = 1)
npStat = np.amax(ds2, axis = axNum)
npStat = np.squeeze(npStat)

In [182]:
a= np.squeeze(stat1.maxCum) - npStat
a.values
#np.squeeze(stat1.minCum.compute())

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
loadPath = "/home/b/b382291/git/data/"
files = os.listdir(loadPath)
files = sorted(files)
savedData = xr.open_dataset(loadPath + files[0])


ds2 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = savedData.meanVar - npMean
a.values


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [17]:
ds2 = tasr.isel(time=slice(24,30)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = meanStat.meanCum - npMean


NameError: name 'tasr' is not defined

In [39]:
try:
    getattr(tasr, "var")
except AttributeError:
    b = 4

In [6]:
# works but really slow 
a = np.array(ds[0,:,:])
b = np.array(ds[1,:,:])

In [5]:
# doesn't work because it's dask 
c = xr.DataArray(ds[1,:,:])
d = xr.DataArray(ds[0,:,:])

In [55]:
# works but really slow 
e = (a < b)
e = e.astype(np.int8)

a[e] = b[e]


In [15]:
# doesn't work 
#f = c < d
#f = f.astype(np.int8)

#np.take(d, f)
#c[f] = d[f]

a = np.array([4, 5, 6, 7],[1,8,9,13])

def _complex_operation(my_arr1, my_arr2):
    f = my_arr1 < my_arr2
    f = f.astype(np.int8)
    my_arr1[f] = my_arr2[f]
    return f 

delayed_result = dask.delayed(_complex_operation)(c, d)
# to create a dask array to use in the future
my_new_arr = da.from_delayed(delayed_result, dtype=c.dtype, shape=c.shape)


TypeError: Field elements must be 2- or 3-tuples, got '1'

In [75]:
c1 = xr.DataArray(np.random.rand(900,1800))
d1 = xr.DataArray(np.random.rand(900,1800))
f1 = (c1 < d1)
f1 = f1.astype(np.int8)
c1[f1] = d1[f1]

In [79]:
c

<xarray.DataArray '10u' (lat: 900, lon: 1800)>
dask.array<getitem, shape=(900, 1800), dtype=float64, chunksize=(900, 1800), chunktype=numpy.ndarray>
Coordinates:
    time     datetime64[ns] 2020-01-20T23:00:00
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
Attributes: (12/16)
    paramId:                    165
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    cfVarName:                  u10
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid
    OPA:                        daily_mean_calculated using one-pass algorithm

In [53]:
#a = np.array([[[4, 5, 6, 7],[1,10,9,13]],[[0,0,4,0],[0,0,4,0]]])#
#b = np.array([[[8, 8, 8, 2],[1,7,10,13]],[[0,3,0,1],[0,3,0,1]]])
#a = da.from_array(a)
#b = da.from_array(b)

a = xr.DataArray(np.random.rand(1,900,1800))
b = np.random.rand(1,900,1800)
#a = da.from_array(a)
#b = da.from_array(b)

f = (da.asarray(ds))
e = (da.asarray(stat1.minCum))


def _complex_operation(my_arr1, my_arr2):
    fIndex = my_arr1 < my_arr2
    fIndex = fIndex.astype(np.int8)
    my_arr1[fIndex==0] = my_arr2[fIndex==0]
    return my_arr1 


tempOut = _complex_operation(a,b)

#delayed_result = dask.delayed(_complex_operation)(f,e)
# to create a dask array to use in the future
#my_new_arr = da.from_delayed(delayed_result, dtype=ds.dtype, shape=e.shape)
#cat = my_new_arr.compute()

IndexError: 3-dimensional boolean indexing is not supported. 

In [149]:

for i in range(0, 24, 1): 
    ds = tasr.isel(time=slice(i,i+1)) # slice(start, stop, step), extract 'moving window' which is hourly data
    
    if (i > 0):
        minCum['time'] = ds.time

#a = np.array([[[-4, -5, 6, 7],[1,10,9,-13]],[[0,0,-4,0],[0,0,-4,0]]])#
#my_arr2 = xr.DataArray([[[-8, -9, 8, -2],[1,7,1,-23]],[[0,3,0,1],[0,3,0,1]]])
        ds = ds.where(ds < minCum, minCum)

    minCum = ds



In [154]:
ds = xr.DataArray([[[-19, -9, 8, -2],[1,7,1,-2]],[[0,3,10,1],[0,3,0,1]]])

ds = ds.where(ds < minCum, minCum)

minCum = ds


In [155]:
minCum

<xarray.DataArray (dim_0: 2, dim_1: 2, dim_2: 4)>
array([[[-19,  -9,   8,  -2],
        [  1,   7,   1, -23]],

       [[  0,   3,   0,   1],
        [  0,   3,   0,   1]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [225]:
60/17

3.5294117647058822

In [224]:
stat1.timeStamp.minute

0

In [214]:


if (isinstance(60/35, int)):

    self.nData = int(self.statFreqMin/self.timeStep) # number of elements of data that need to be added to the cumulative mean
else: 
    # we have a problem 
    print('Frequency of the requested statistic (e.g. daily) must be wholly divisable by the timestep (dt) of the input data')


Frequency of the statistic must be wholly divisable by the timestep


In [151]:
ds1 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data
a = np.min(ds1, axis = 0)
a.values

array([[-3.19230318, -3.19230318, -3.19230318, ..., -3.19230318,
        -3.19230318, -3.19230318],
       [-3.64486769, -3.64486769, -3.64486769, ..., -3.64486769,
        -3.64486769, -3.64486769],
       [-4.35189571, -4.35189571, -4.35189571, ..., -4.35189571,
        -4.35189571, -4.35189571],
       ...,
       [-4.30393076, -4.30393076, -4.30393076, ..., -4.30393076,
        -4.30393076, -4.30393076],
       [-4.47970805, -4.47970805, -4.47970805, ..., -4.47970805,
        -4.47970805, -4.47970805],
       [-4.78157358, -4.78157358, -4.78157358, ..., -4.78157358,
        -4.78157358, -4.78157358]])

In [187]:
ds1 = tasr.isel(time=slice(0,100)) # slice(start, stop, step), extract 'moving window' which is hourly data
a = np.mean(ds1, axis = 0)
b = np.mean(a, axis = 0)
c = np.mean(b, axis =0)

In [189]:
b.values

array([-0.33142749, -0.31430956, -0.32023877, ..., -0.33465021,
       -0.3385892 , -0.34482014])